In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from trainer.RetrievalTrainer import trainer
from datasets.ScannetDataset import ScannetDataset
from datasets.CategoryTestTimeDataset import *
from datasets.Reader import *

from test_time.FeatureExtractor import FeatureExtractor
from test_time.RetrievalModule import RetrievalModule

from utils.logger import logger
from utils.ckpts import load_checkpoint, save_checkpoint

from utils.retrieval import *
from utils.visualize import *
from utils.preprocess import random_rotation, load_norm_pc, apply_transform
from utils.symmetry import *
from utils.eval_pose import *
from utils.Info.Scan2cadInfo import Scan2cadInfo
from utils.Info.CADLib import *

from model import load_model, fc

from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
import matplotlib
import time

import MinkowskiEngine as ME

class Config():
    def __init__(self, category):

        self.root = "/scannet/ShapeNetCore.v2.PC15k"
        self.scan2cad_root = "/scannet/crop_scan2cad_filter/data"
        self.cad_root = "/scannet/ShapeNetCore.v2.PC15k"
        
        if category == "chair":
            self.catid = "03001627"
        elif category == "table":
            self.catid = "04379243"
        else:
            self.catid = ""
        self.voxel_size = 0.03
        #self.dim = [1024, 512,  256]
        #self.embedding = "identity"
        if category == "chair":
            #self.resume = "./ckpts/scannet_ret_max_sim0"
            self.resume = "./ckpts/scannet_ret_chair"
        elif category == "table":
            #self.resume = "./ckpts/scannet_table_pose_001_usedLib_bestMatch_Best"
            self.resume = "./ckpts/scannet_ret_table"
        else:
            self.resume = ""

        self.model = "ResUNetBN2C"
        self.model_n_out = 16
        self.normalize_feature = True
        self.conv1_kernel_size = 3
        self.bn_momentum = 0.05
        self.nn_max_n = 500
        self.lib_path = "./CadLib"
        self.scan2cad_dict = "/scannet/scan2cad_download_link/unique_cads.csv"
        self.annotation_dir = "/scannet/scan2cad_download_link"
        self.device = torch.device("cuda")
        self.retrieve_cad = True

def chamfer_gpu(pc0, pc1):
    pc0 = pc0[None,:,:]
    pc1 = pc1[:,None,:]
    delta = pc0 - pc1

    return delta.norm(dim=2).min(0)[0].mean() + delta.norm(dim=2).min(1)[0].mean()


def chamfer_gpu_1direction(pc0, pc1):

    pc0 = pc0[None,:,:]
    pc1 = pc1[:,None,:]
    delta = pc0 - pc1
    return delta.norm(dim=2).min(0)[0].mean().item()


def eval_lib(model, embedding, cadlib_loader, distance):
    np.random.seed(31)
    torch.random.manual_seed(31)
    model.eval()
    embedding.eval()

    lib_feats = []
    lib_outputs = []
    lib_Ts = []
    lib_origins = []

    with torch.no_grad():
        print("Updating global feature in the CAD Lib")
        for idx, data in enumerate(cadlib_loader):
            print("Lib feature Batch: {}/{}".format(idx+1, len(cadlib_loader)))

            base_input = ME.SparseTensor(data["base_feat"], data["base_coords"]).to("cuda")

            lib_Ts.append(data["base_T"])
            batch_size = len(data["base_idx"])

            base_output, base_feat = model(base_input)
            base_feat = embedding(base_feat)

            for i in range(batch_size):
                base_mask = base_output.C[:,0]==i

                lib_outputs.append(base_output.F[base_mask, :])
                lib_origins.append(data["base_origin"][base_mask, :])

            if distance == "l2":
                base_feat_norm = F.normalize(base_feat, dim=1) 
                lib_feats.append(base_feat_norm)

            else:
                base_feat_norm = F.normalize(base_feat.F, dim=1)

                for i in range(batch_size):
                    base_feat_mask = base_feat.C[:,0] == i
                    lib_feats.append(base_feat_norm[base_feat_mask, :])  


    lib_Ts = torch.cat(lib_Ts, 0)

    return lib_feats, lib_outputs, lib_origins, lib_Ts

In [2]:
config = Config("table")

logger = logger("./logs", "debug_scan2cad_eval.txt")
#logger = logger("./logs", "scan2cad-test-{}-{}-log.txt".format(config.catid, config.retrieve_cad))

logger.log("catid: {}".format(config.catid))
logger.log("use retrieve cad: {}".format(config.retrieve_cad))
logger.log("resume: {}".format(config.resume))


num_feats = 1
Model = load_model("ResUNetBN2C")
model = Model(
    num_feats,
    16,
    bn_momentum=0.05,
    normalize_feature=True,
    conv1_kernel_size=3,
    D=3)
model = model.to(torch.device("cuda"))


# Embedding network for retrieval
distance = "l2"#"chamfer"
embedding = fc.conv1_max_embedding(1024, 512, 256).to("cuda")
#embedding = fc.conv1_fc_chamfer(1024, 512, 256).to("cuda")

checkpoint = torch.load(config.resume)
model.load_state_dict(checkpoint["state_dict"])
embedding.load_state_dict(checkpoint["embedding_state_dict"])

logger.log("ckpt epoch: {}".format(checkpoint["epoch"]))

model.eval()
embedding.eval()

2021-03-28 15:57:52 catid: 04379243
2021-03-28 15:57:52 use retrieve cad: True
2021-03-28 15:57:52 resume: ./ckpts/scannet_ret_table
2021-03-28 15:57:57 ckpt epoch: 99


conv1_max_embedding(
  (final): conv1_chamfer(
    (final): MinkowskiConvolution(in=256, out=1024, region_type=RegionType.HYPERCUBE, kernel_size=[1, 1, 1], stride=[1, 1, 1], dilation=[1, 1, 1])
  )
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)

In [4]:
# random seed
np.random.seed(31)
torch.random.manual_seed(31)

# dataset
scan2cad_info = Scan2cadInfo(config.cad_root, config.scan2cad_root, config.catid, config.annotation_dir)

table_path = "/scannet/tables/{}_scan2cad.npy".format(config.catid)
cadlib = CustomizeCADLib(config.cad_root, config.catid, scan2cad_info.UsedObjId, table_path, 
                              config.voxel_size, False)

cadlib_loader = torch.utils.data.DataLoader(cadlib, batch_size=32, 
                                        shuffle=False, num_workers=4, collate_fn=cadlib.collate_pair_fn)

dataset = ScannetDataset(config.scan2cad_root, config.cad_root, cadlib, scan2cad_info, "test", config.catid, 
                                            0.0, 0.5, 0.03, preload=False)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, 
                                        shuffle=False, num_workers=2, collate_fn=dataset.collate_pair_fn)

# load sym label

with open("./config/{}_{}_rot_sym_label.txt".format(config.catid, "scan2cad"), "r") as f:

    lines = f.readlines()

    names = [line.strip('\n').split(' ')[0] for line in lines]
    sym_ref = [int(line.strip('\n').split(' ')[1]) for line in lines]

sym_label = sym_ref

# cad models feature extraction
lib_feats, lib_outputs, lib_origins, lib_Ts  = eval_lib(model, embedding, cadlib_loader, distance)

logger.log("lib size: {}".format(len(lib_feats)))

0.03
Loading Shapenet CAD models
Loading Scannet scans
Updating global feature in the CAD Lib
Lib feature Batch: 1/26
Lib feature Batch: 2/26
Lib feature Batch: 3/26
Lib feature Batch: 4/26
Lib feature Batch: 5/26
Lib feature Batch: 6/26
Lib feature Batch: 7/26
Lib feature Batch: 8/26
Lib feature Batch: 9/26
Lib feature Batch: 10/26
Lib feature Batch: 11/26
Lib feature Batch: 12/26
Lib feature Batch: 13/26
Lib feature Batch: 14/26
Lib feature Batch: 15/26
Lib feature Batch: 16/26
Lib feature Batch: 17/26
Lib feature Batch: 18/26
Lib feature Batch: 19/26
Lib feature Batch: 20/26
Lib feature Batch: 21/26
Lib feature Batch: 22/26
Lib feature Batch: 23/26
Lib feature Batch: 24/26
Lib feature Batch: 25/26
Lib feature Batch: 26/26
2021-03-25 15:12:51 lib size: 26


In [5]:
# scans feature extraction
with torch.no_grad():
    np.random.seed(31)
    torch.random.manual_seed(31)
    model.eval()
    embedding.eval()

    t_loss_avg = 0
    r_loss_avg = 0

    pos_idx = []

    base_outputs = []#;pos_outputs = []
    base_origins = []#;pos_origins = []

    base_feats = []#;pos_feats = []

    base_Ts = []#;pos_Ts = []
    
    pos_syms = []

    num_data = len(dataset)

    # Collect Lib features

    with torch.no_grad():

        for idx, data in enumerate(loader):
            logger.log("Eval feature Batch: {}/{}".format(idx+1, len(loader)))

            base_input = ME.SparseTensor(data["base_feat"], data["base_coords"]).to("cuda")
            #pos_input = ME.SparseTensor(data["pos_feat"], data["pos_coords"]).to("cuda")

            base_Ts.append(data["base_T"])
            #pos_Ts.append(data["pos_T"])
            pos_idx.append(data["pos_idx"])
            pos_syms.append(data["pos_sym"])
            batch_size = len(data["pos_idx"])

            base_output, base_feat = model(base_input)

            #pos_output, pos_feat = self.model(pos_input)

            base_feat = embedding(base_feat)

            for i in range(batch_size):
                base_mask = base_output.C[:,0]==i
                #pos_mask = pos_output.C[:,0]==i

                base_outputs.append(base_output.F[base_mask, :])
                #pos_outputs.append(pos_output.F[pos_mask, :])
                base_origins.append(data["base_origin"][base_mask, :])
                #pos_origins.append(data["pos_origin"][pos_mask, :])


            if distance == "l2":
                base_feat_norm = F.normalize(base_feat, dim=1) #base_feat/base_feat.norm(dim=1, keepdim=True)
                base_feats.append(base_feat_norm)

            else:
                base_feat_norm = F.normalize(base_feat.F, dim=1) #base_feat/base_feat.norm(dim=1, keepdim=True)

                for i in range(batch_size):
                    base_feat_mask = base_feat.C[:,0] == i
                    base_feats.append(base_feat_norm[base_feat_mask, :])  

        pos_idx = torch.cat(pos_idx, 0)
        base_Ts = torch.cat(base_Ts, 0)
        pos_syms = torch.cat(pos_syms, 0)

# retrieval part
if distance == "chamfer":
    dists = np.zeros([len(base_feats), len(lib_feats)])
    for i in range(len(base_feats)):
        for j in range(len(lib_feats)):
            dists[i,j] = self.chamfer_gpu(base_feats[i], lib_feats[j])
    #dists += dists.T
    BestMatchIdx = pos_idx.detach().cpu().numpy()

    stat = scan2cad_retrieval_eval_dist(dists, dataset.table, BestMatchIdx, max(dataset.pos_n, 1))
    self.logger.log("top1_error: {} percision: {}".format(stat["top1_error"], stat["percision"]))
else:
    descriptor = torch.cat(base_feats, 0).detach().cpu().numpy()
    lib_feats = torch.cat(lib_feats, 0).detach().cpu().numpy()

    BestMatchIdx = pos_idx.detach().cpu().numpy()
    stat = scan2cad_retrieval_eval(descriptor, lib_feats, BestMatchIdx, dataset.table, max(dataset.pos_n, 1))
    print("top1_error: {} percision: {}".format(stat["top1_error"], stat["percision"]))

2021-03-25 15:13:05 Eval feature Batch: 1/10
2021-03-25 15:13:07 Eval feature Batch: 2/10
2021-03-25 15:13:22 Eval feature Batch: 3/10
2021-03-25 15:13:24 Eval feature Batch: 4/10
2021-03-25 15:13:40 Eval feature Batch: 5/10
2021-03-25 15:13:43 Eval feature Batch: 6/10
2021-03-25 15:13:57 Eval feature Batch: 7/10
2021-03-25 15:14:00 Eval feature Batch: 8/10
2021-03-25 15:14:12 Eval feature Batch: 9/10
2021-03-25 15:14:14 Eval feature Batch: 10/10
top1_error: 0.1906090742201125 percision: 4.123711340206185


In [6]:
gt = stat["gt"]
top1_predict = stat["top1_predict"]

# pose estimation
if config.retrieve_cad:
    select = top1_predict
    logger.log("Using top-1 prediction")
else:
    select = gt
    logger.log("Using annotation")

k_nn=5
max_corr = 0.20

with torch.no_grad():
    t_losses_ransac = []
    r_losses_ransac = []

    t_losses_sym = []
    r_losses_sym = []
    
    chamf_ransac = []
    chamf_sym = []
    
    
    for idx in range(len(dataset)):
        logger.log("----------------------")
        if (idx+1) % 10 == 0:
            
            logger.log("Eval align Index: {}/{}  ".format(idx+1, len(dataset))) 
                            
            logger.log("RANSAC: T error: {:.4f} R error: {:.4f}".format(sum(t_losses_ransac)/len(t_losses_ransac), 
                                                           sum(r_losses_ransac)/len(r_losses_ransac)))
            
            logger.log("SYM: T error: {:.4f} R error: {:.4f}".format(sum(t_losses_sym)/len(t_losses_sym), 
                                                           sum(r_losses_sym)/len(r_losses_sym)))
            logger.log("----------------------")

        # matching based pose estimation
        xyz0, xyz1, T0, T1 = base_origins[idx], lib_origins[select[idx]], base_Ts[idx,:,:], lib_Ts[select[idx],:,:]
        baseF, posF = base_outputs[idx], lib_outputs[select[idx]]

        pos_sym = sym_label[select[idx]]
        
        ####################################################################################################################
        
        T_est_best, chamf_dist_best, T_est_ransac, chamf_dist_ransac = sym_pose(baseF, xyz0, posF, 
                                                                                xyz1, pos_sym, k_nn, max_corr)
        
        t_loss_sym, r_loss_sym = eval_pose(T_est_best, T0, T1, axis_symmetry=pos_sym)

        t_loss_ransac, r_loss_ransac = eval_pose(T_est_ransac, T0, T1, axis_symmetry=pos_sym)
        
        t_losses_ransac.append(t_loss_ransac)
        r_losses_ransac.append(r_loss_ransac)
        chamf_ransac.append(chamf_dist_ransac)

        logger.log("ransac: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(t_loss_ransac, 
                                                                 r_loss_ransac, 
                                                                 chamf_dist_ransac ))
        
        logger.log("ransac avg: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(sum(t_losses_ransac)/len(t_losses_ransac), 
                                                                     sum(r_losses_ransac)/len(r_losses_ransac),
                                                                     sum(chamf_ransac)/len(chamf_ransac)))
        
            
        t_losses_sym.append(t_loss_sym)
        r_losses_sym.append(r_loss_sym)
        chamf_sym.append(chamf_dist_best)

        logger.log("sym: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(t_loss_sym, 
                                                              r_loss_sym, 
                                                              chamf_dist_best))
        
        logger.log("sym avg: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(sum(t_losses_sym)/len(t_losses_sym), 
                                                                  sum(r_losses_sym)/len(r_losses_sym),
                                                                  sum(chamf_sym)/len(chamf_sym)))
        logger.log("----------------------")

            

2021-03-25 15:14:16 Using top-1 prediction
2021-03-25 15:14:16 ----------------------
2021-03-25 15:14:24 ransac: t: 0.1370 r: 0.3193 chamf: 0.0706
2021-03-25 15:14:24 ransac avg: t: 0.1370 r: 0.3193 chamf: 0.0706
2021-03-25 15:14:24 sym: t: 0.3373 r: 0.2856 chamf: 0.0552
2021-03-25 15:14:24 sym avg: t: 0.3373 r: 0.2856 chamf: 0.0552
2021-03-25 15:14:24 ----------------------
2021-03-25 15:14:24 ----------------------
2021-03-25 15:14:35 ransac: t: 0.1219 r: 0.0909 chamf: 0.0587
2021-03-25 15:14:35 ransac avg: t: 0.1295 r: 0.2051 chamf: 0.0646
2021-03-25 15:14:35 sym: t: 0.0654 r: 0.0139 chamf: 0.0522
2021-03-25 15:14:35 sym avg: t: 0.2013 r: 0.1497 chamf: 0.0537
2021-03-25 15:14:35 ----------------------
2021-03-25 15:14:35 ----------------------
2021-03-25 15:14:40 ransac: t: 1.3067 r: 3.1295 chamf: 0.0518
2021-03-25 15:14:40 ransac avg: t: 0.5219 r: 1.1799 chamf: 0.0603
2021-03-25 15:14:40 sym: t: 1.3052 r: 3.1019 chamf: 0.0493
2021-03-25 15:14:40 sym avg: t: 0.5693 r: 1.1338 chamf:

2021-03-25 15:17:37 ransac: t: 0.8114 r: 3.1174 chamf: 0.0736
2021-03-25 15:17:37 ransac avg: t: 0.4694 r: 1.5862 chamf: 0.0801
2021-03-25 15:17:37 sym: t: 0.0786 r: 0.0191 chamf: 0.0441
2021-03-25 15:17:37 sym avg: t: 0.3268 r: 0.7911 chamf: 0.0670
2021-03-25 15:17:37 ----------------------
2021-03-25 15:17:37 ----------------------
2021-03-25 15:17:50 ransac: t: 0.7208 r: 3.0665 chamf: 0.0888
2021-03-25 15:17:50 ransac avg: t: 0.4795 r: 1.6455 chamf: 0.0804
2021-03-25 15:17:50 sym: t: 0.7119 r: 3.0476 chamf: 0.0691
2021-03-25 15:17:50 sym avg: t: 0.3422 r: 0.8813 chamf: 0.0671
2021-03-25 15:17:50 ----------------------
2021-03-25 15:17:50 ----------------------
2021-03-25 15:18:00 ransac: t: 0.0332 r: 0.0591 chamf: 0.0310
2021-03-25 15:18:00 ransac avg: t: 0.4623 r: 1.5844 chamf: 0.0785
2021-03-25 15:18:00 sym: t: 0.0332 r: 0.0591 chamf: 0.0310
2021-03-25 15:18:00 sym avg: t: 0.3303 r: 0.8497 chamf: 0.0657
2021-03-25 15:18:00 ----------------------
2021-03-25 15:18:00 ---------------

2021-03-25 15:21:56 ransac: t: 1.0459 r: 1.6498 chamf: 0.0608
2021-03-25 15:21:56 ransac avg: t: 0.4025 r: 1.1483 chamf: 0.0781
2021-03-25 15:21:56 sym: t: 1.0268 r: 1.6351 chamf: 0.0598
2021-03-25 15:21:56 sym avg: t: 0.2950 r: 0.6552 chamf: 0.0631
2021-03-25 15:21:56 ----------------------
2021-03-25 15:21:56 ----------------------
2021-03-25 15:22:01 ransac: t: 0.0303 r: 0.1159 chamf: 0.0436
2021-03-25 15:22:01 ransac avg: t: 0.3949 r: 1.1273 chamf: 0.0774
2021-03-25 15:22:01 sym: t: 0.0704 r: 0.1441 chamf: 0.0387
2021-03-25 15:22:01 sym avg: t: 0.2904 r: 0.6448 chamf: 0.0626
2021-03-25 15:22:01 ----------------------
2021-03-25 15:22:01 ----------------------
2021-03-25 15:22:01 Eval align Index: 50/291  
2021-03-25 15:22:01 RANSAC: T error: 0.3949 R error: 1.1273
2021-03-25 15:22:01 SYM: T error: 0.2904 R error: 0.6448
2021-03-25 15:22:01 ----------------------
2021-03-25 15:22:07 ransac: t: 0.9812 r: 1.6392 chamf: 0.0749
2021-03-25 15:22:07 ransac avg: t: 0.4066 r: 1.1375 chamf: 

2021-03-25 15:25:43 ransac: t: 0.1508 r: 0.1378 chamf: 0.1100
2021-03-25 15:25:43 ransac avg: t: 0.3738 r: 1.0560 chamf: 0.0761
2021-03-25 15:25:43 sym: t: 0.1575 r: 0.1970 chamf: 0.0945
2021-03-25 15:25:43 sym avg: t: 0.2741 r: 0.6333 chamf: 0.0620
2021-03-25 15:25:43 ----------------------
2021-03-25 15:25:43 ----------------------
2021-03-25 15:25:53 ransac: t: 0.1985 r: 0.0251 chamf: 0.0616
2021-03-25 15:25:53 ransac avg: t: 0.3714 r: 1.0417 chamf: 0.0759
2021-03-25 15:25:53 sym: t: 0.1985 r: 0.0251 chamf: 0.0616
2021-03-25 15:25:53 sym avg: t: 0.2730 r: 0.6248 chamf: 0.0620
2021-03-25 15:25:53 ----------------------
2021-03-25 15:25:53 ----------------------
2021-03-25 15:25:59 ransac: t: 0.1207 r: 0.0969 chamf: 0.0520
2021-03-25 15:25:59 ransac avg: t: 0.3680 r: 1.0288 chamf: 0.0756
2021-03-25 15:25:59 sym: t: 0.2023 r: 0.1505 chamf: 0.0484
2021-03-25 15:25:59 sym avg: t: 0.2720 r: 0.6183 chamf: 0.0618
2021-03-25 15:25:59 ----------------------
2021-03-25 15:25:59 ---------------

2021-03-25 15:29:30 ransac: t: 0.0608 r: 0.1518 chamf: 0.1000
2021-03-25 15:29:30 ransac avg: t: 0.3652 r: 1.0388 chamf: 0.0800
2021-03-25 15:29:30 sym: t: 0.0967 r: 0.0650 chamf: 0.0682
2021-03-25 15:29:30 sym avg: t: 0.2781 r: 0.6383 chamf: 0.0645
2021-03-25 15:29:30 ----------------------
2021-03-25 15:29:30 ----------------------
2021-03-25 15:29:45 ransac: t: 0.3009 r: 0.1852 chamf: 0.1898
2021-03-25 15:29:45 ransac avg: t: 0.3646 r: 1.0299 chamf: 0.0811
2021-03-25 15:29:45 sym: t: 0.2475 r: 0.1679 chamf: 0.1531
2021-03-25 15:29:45 sym avg: t: 0.2778 r: 0.6334 chamf: 0.0654
2021-03-25 15:29:45 ----------------------
2021-03-25 15:29:45 ----------------------
2021-03-25 15:30:00 ransac: t: 0.4789 r: 0.9541 chamf: 0.1470
2021-03-25 15:30:00 ransac avg: t: 0.3657 r: 1.0291 chamf: 0.0818
2021-03-25 15:30:00 sym: t: 0.5002 r: 0.7976 chamf: 0.1159
2021-03-25 15:30:00 sym avg: t: 0.2801 r: 0.6351 chamf: 0.0659
2021-03-25 15:30:00 ----------------------
2021-03-25 15:30:00 ---------------

...

2021-03-25 15:44:37 ransac: t: 0.0868 r: 0.1802 chamf: 0.0531
2021-03-25 15:44:37 ransac avg: t: 0.3760 r: 1.0761 chamf: 0.0859
2021-03-25 15:44:37 sym: t: 0.1426 r: 0.2158 chamf: 0.0464
2021-03-25 15:44:37 sym avg: t: 0.3059 r: 0.8384 chamf: 0.0697
2021-03-25 15:44:37 ----------------------
2021-03-25 15:44:37 ----------------------
2021-03-25 15:44:51 ransac: t: 0.9474 r: 2.0010 chamf: 0.1957
2021-03-25 15:44:54 ransac avg: t: 0.3790 r: 1.0809 chamf: 0.0864
2021-03-25 15:44:54 sym: t: 0.9984 r: 2.9067 chamf: 0.1450
2021-03-25 15:44:54 sym avg: t: 0.3095 r: 0.8492 chamf: 0.0701
2021-03-25 15:44:54 ----------------------
2021-03-25 15:44:54 ----------------------
2021-03-25 15:45:03 ransac: t: 0.0575 r: 0.0516 chamf: 0.0657
2021-03-25 15:45:03 ransac avg: t: 0.3774 r: 1.0756 chamf: 0.0863
2021-03-25 15:45:03 sym: t: 0.0606 r: 0.0262 chamf: 0.0609
2021-03-25 15:45:03 sym avg: t: 0.3082 r: 0.8449 chamf: 0.0701
2021-03-25 15:45:03 ----------------------
2021-03-25 15:45:03 ---------------

2021-03-25 15:48:30 ransac: t: 0.2232 r: 0.1708 chamf: 0.0558
2021-03-25 15:48:30 ransac avg: t: 0.3993 r: 1.1316 chamf: 0.0865
2021-03-25 15:48:30 sym: t: 0.1220 r: 0.1409 chamf: 0.0459
2021-03-25 15:48:30 sym avg: t: 0.3181 r: 0.8665 chamf: 0.0706
2021-03-25 15:48:30 ----------------------
2021-03-25 15:48:30 ----------------------
2021-03-25 15:48:41 ransac: t: 0.0968 r: 0.0640 chamf: 0.0619
2021-03-25 15:48:41 ransac avg: t: 0.3979 r: 1.1267 chamf: 0.0864
2021-03-25 15:48:41 sym: t: 0.0750 r: 0.1179 chamf: 0.0562
2021-03-25 15:48:41 sym avg: t: 0.3170 r: 0.8630 chamf: 0.0706
2021-03-25 15:48:41 ----------------------
2021-03-25 15:48:41 ----------------------
2021-03-25 15:48:47 ransac: t: 0.9474 r: 2.9208 chamf: 0.1004
2021-03-25 15:48:47 ransac avg: t: 0.4004 r: 1.1350 chamf: 0.0864
2021-03-25 15:48:47 sym: t: 0.8922 r: 3.0548 chamf: 0.0837
2021-03-25 15:48:47 sym avg: t: 0.3197 r: 0.8731 chamf: 0.0706
2021-03-25 15:48:47 ----------------------
2021-03-25 15:48:47 ---------------

2021-03-25 15:52:07 ransac: t: 0.2932 r: 0.0754 chamf: 0.0777
2021-03-25 15:52:07 ransac avg: t: 0.3949 r: 1.1271 chamf: 0.0859
2021-03-25 15:52:07 sym: t: 0.2932 r: 0.0754 chamf: 0.0777
2021-03-25 15:52:07 sym avg: t: 0.3165 r: 0.8790 chamf: 0.0706
2021-03-25 15:52:07 ----------------------
2021-03-25 15:52:07 ----------------------
2021-03-25 15:52:17 ransac: t: 0.5880 r: 0.1553 chamf: 0.0471
2021-03-25 15:52:17 ransac avg: t: 0.3957 r: 1.1230 chamf: 0.0857
2021-03-25 15:52:17 sym: t: 0.6277 r: 0.2491 chamf: 0.0356
2021-03-25 15:52:17 sym avg: t: 0.3178 r: 0.8764 chamf: 0.0704
2021-03-25 15:52:17 ----------------------
2021-03-25 15:52:17 ----------------------
2021-03-25 15:52:17 Eval align Index: 240/291  
2021-03-25 15:52:17 RANSAC: T error: 0.3957 R error: 1.1230
2021-03-25 15:52:17 SYM: T error: 0.3178 R error: 0.8764
2021-03-25 15:52:17 ----------------------
2021-03-25 15:52:28 ransac: t: 1.1960 r: 3.0809 chamf: 0.1199
2021-03-25 15:52:28 ransac avg: t: 0.3991 r: 1.1312 chamf:

2021-03-25 15:55:15 ransac: t: 1.2510 r: 3.1187 chamf: 0.0263
2021-03-25 15:55:15 ransac avg: t: 0.3968 r: 1.1287 chamf: 0.0840
2021-03-25 15:55:15 sym: t: 1.3509 r: 3.0778 chamf: 0.0255
2021-03-25 15:55:15 sym avg: t: 0.3223 r: 0.8822 chamf: 0.0691
2021-03-25 15:55:15 ----------------------
2021-03-25 15:55:15 ----------------------
2021-03-25 15:55:20 ransac: t: 0.4062 r: 2.8431 chamf: 0.0500
2021-03-25 15:55:20 ransac avg: t: 0.3968 r: 1.1352 chamf: 0.0839
2021-03-25 15:55:20 sym: t: 0.4062 r: 2.8431 chamf: 0.0500
2021-03-25 15:55:20 sym avg: t: 0.3227 r: 0.8897 chamf: 0.0690
2021-03-25 15:55:20 ----------------------
2021-03-25 15:55:20 ----------------------
2021-03-25 15:55:28 ransac: t: 0.0705 r: 0.2003 chamf: 0.0491
2021-03-25 15:55:28 ransac avg: t: 0.3956 r: 1.1316 chamf: 0.0838
2021-03-25 15:55:28 sym: t: 0.0421 r: 0.0321 chamf: 0.0371
2021-03-25 15:55:28 sym avg: t: 0.3216 r: 0.8865 chamf: 0.0689
2021-03-25 15:55:28 ----------------------
2021-03-25 15:55:28 ---------------

2021-03-25 15:58:20 ransac: t: 0.0340 r: 0.0472 chamf: 0.0262
2021-03-25 15:58:20 ransac avg: t: 0.4065 r: 1.1515 chamf: 0.0829
2021-03-25 15:58:20 sym: t: 0.0515 r: 0.0674 chamf: 0.0252
2021-03-25 15:58:20 sym avg: t: 0.3280 r: 0.8920 chamf: 0.0680
2021-03-25 15:58:20 ----------------------
2021-03-25 15:58:20 ----------------------
2021-03-25 15:58:33 ransac: t: 0.2916 r: 0.1303 chamf: 0.0650
2021-03-25 15:58:33 ransac avg: t: 0.4061 r: 1.1479 chamf: 0.0828
2021-03-25 15:58:33 sym: t: 0.2633 r: 0.1048 chamf: 0.0598
2021-03-25 15:58:33 sym avg: t: 0.3277 r: 0.8893 chamf: 0.0680
2021-03-25 15:58:33 ----------------------
2021-03-25 15:58:33 ----------------------
2021-03-25 15:58:48 ransac: t: 1.6792 r: 3.1134 chamf: 0.1949
2021-03-25 15:58:48 ransac avg: t: 0.4105 r: 1.1547 chamf: 0.0832
2021-03-25 15:58:48 sym: t: 0.3691 r: 0.1516 chamf: 0.0906
2021-03-25 15:58:48 sym avg: t: 0.3279 r: 0.8867 chamf: 0.0681
2021-03-25 15:58:48 ----------------------
2021-03-25 15:58:48 ---------------

In [7]:
"""
np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-T-ransac.npy".format(config.catid, config.retrieve_cad), 
                                                                         np.array(t_losses_ransac))
np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-R-ransac.npy".format(config.catid, config.retrieve_cad),
                                                                        np.array(r_losses_ransac))

np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-T-sym.npy".format(config.catid, config.retrieve_cad),
                                                                        np.array(t_losses_sym))
np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-R-sym.npy".format(config.catid, config.retrieve_cad),
                                                                        np.array(r_losses_sym))

np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-chamf-ransac.npy".format(config.catid, config.retrieve_cad),
                                                                        np.array(chamf_ransac))
np.save("/zty-vol/results/stats/scan2cad-test-{}-{}-chamf-sym.npy".format(config.catid, config.retrieve_cad),
                                                                        np.array(chamf_sym))
"""

logger.log("ransac avg: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(sum(t_losses_ransac)/len(t_losses_ransac), 
                                                             sum(r_losses_ransac)/len(r_losses_ransac),
                                                             sum(chamf_ransac)/len(chamf_ransac)))

logger.log("sym avg: t: {:.4f} r: {:.4f} chamf: {:.4f}".format(sum(t_losses_sym)/len(t_losses_sym), 
                                                                  sum(r_losses_sym)/len(r_losses_sym),
                                                                  sum(chamf_sym)/len(chamf_sym)))

2021-03-25 15:59:40 ransac avg: t: 0.4068 r: 1.1424 chamf: 0.0834
2021-03-25 15:59:40 sym avg: t: 0.3274 r: 0.8970 chamf: 0.0682
